## **GoogLeNet very deep convolutional Model**

### **1.Installing and importing requirements**

In [ ]:
! pip install "ray[tune]" torch torchvision pytorch-lightning
'''
Ray Tune is a Python library that accelerates hyperparameter tuning by allowing
you to leverage cutting edge optimization algorithms at scale.

PyTorch Lighting is a lightweight PyTorch wrapper for high-performance AI research
that aims to abstract Deep Learning boilerplate while providing you full control
and flexibility over your code 

'''
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl
'''
preparing to use the google cloud tpu capabilities
'''

!nvidia-smi # checking nvidia system managment interface

Ali


In [ ]:
import torch

import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision

from torchvision import transforms
from torchvision.transforms import ToTensor


from torch import nn

from torchvision import datasets
from torchvision.datasets import CIFAR10

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
import gc

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm.autonotebook import tqdm
from sklearn.metrics import classification_report

In [ ]:
print(torch.__version__)
print(pl.__version__)

### **2.HyperParameters**


In [ ]:
# Hyperparameters
RANDOM_SEED = 42
LEARNING_RATE = 1e-2
BATCH_SIZE = 2
NUM_EPOCHS = 10

# Architecture
NUM_FEATURES = 28*28
NUM_CLASSES = 10

# Other
DEVICE = "cuda:0"
GRAYSCALE = False

In [ ]:
torch.manual_seed(RANDOM_SEED)

In [ ]:
transform = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

train_set = datasets.CIFAR10(root='data', 
                               train=True, 
                               transform=transform,
                               download=True)

test_set = datasets.CIFAR10(root='data', 
                              train=False, 
                              transform=transform)


train_loader = DataLoader(dataset=train_set, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_set, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)
images, labels = next(iter(train_loader):  
print('Image batch dimensions:', images.shape)
print('Image label dimensions:', labels.shape)


In [ ]:
images , _ = next(iter(train_loader))
print('images.shape:', images.shape)
plt.figure(figsize=(16,8))
plt.axis('off')
plt.imshow(torchvision.utils.make_grid(images, nrow=16).permute((1, 2, 0))

### **3.Model**


#### **Building small blocks for the Inception Model**


In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, k, s, p):
        super(ConvBlock, self).__init__()
        self.convolution = nn.Sequential(
            nn.Conv2d(in_channels=in_ch,
                      out_channels=out_ch,
                      kernel_size=(k, k),
                      stride=(s, s),
                      padding=(p, p)),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.convolution(x)
        return x


class ReducedConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch1,out_ch2, k, p):
        super(ReducedConvBlock, self).__init__()
        self.reducedConv = nn.Sequential(
            nn.Conv2d(in_channels=in_ch,
                      out_channels= out_ch1,
                      kernel_size=(1,1),
                      stride=(1,1)),
            nn.ReLU(),
            nn.Conv2d(in_channels= out_ch1,
                      out_channels= out_ch2,
                      kernel_size=(k, k),
                      stride=(1, 1),
                      padding=(p, p)),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.reducedConv(x)
        return x

class InceptionModule(nn.Module):
    def __init__(self, curr_in_fts, f_1x1, f_3x3_r, f_3x3, f_5x5_r, f_5x5, f_pool_proj):
        super(InceptionModule, self).__init__()
        self.conv1 = ConvBlock(curr_in_fts, f_1x1, 1, 1, 0)
        self.conv2 = ReducedConvBlock(curr_in_fts, f_3x3_r, f_3x3, 3, 1)
        self.conv3 = ReducedConvBlock(curr_in_fts, f_5x5_r, f_5x5, 5, 2)

        self.pool_proj = nn.Sequential(
            nn.MaxPool2d(kernel_size=(1, 1), stride=(1, 1)),
            nn.Conv2d(in_channels=curr_in_fts,
                      out_channels=f_pool_proj,
                      kernel_size=(1, 1),
                      stride=(1, 1)),
            nn.ReLU()
        )

    def forward(self, input_img):
        out1 = self.conv1(input_img)
        out2 = self.conv2(input_img)
        out3 = self.conv3(input_img)
        out4 = self.pool_proj(input_img)


        x = torch.cat([out1, out2, out3, out4], dim=1)

        return x

class AuxClassifier(nn.Module):
    def __init__(self, in_fts, num_classes):
        super(AuxClassifier, self).__init__()
        self.avgpool = nn.AvgPool2d(kernel_size=(5, 5), stride=(3, 3))
        self.conv = nn.Conv2d(in_channels=in_fts,
                              out_channels=128,
                              kernel_size=(1, 1),
                              stride=(1, 1))
        self.relu = nn.ReLU()
        self.fc = nn.Linear(4 * 4 * 128, 1024)
        self.dropout = nn.Dropout(p=0.7)
        self.classifier = nn.Linear(1024, num_classes)

    def forward(self, input_img):
        N = input_img.shape[0]
        x = self.avgpool(input_img)
        x = self.conv(x)
        x = self.relu(x)
        x = x.reshape(N, -1)
        x = self.fc(x)
        x = self.dropout(x)
        x = self.classifier(x)
        return x

**GoogLeNet**

In [ ]:
class GoogleNet(nn.Module):
    def __init__(self, in_fts=3, num_class=10):
        super(GoogleNet, self).__init__()
        self.conv1 = ConvBlock(in_fts, 64, 7, 2, 3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2 = nn.Sequential(
            ConvBlock(64, 64, 1, 1, 0),
            ConvBlock(64, 192, 3, 1, 1)
        )

        self.inception_3a = InceptionModule(192, 64, 96, 128, 16, 32, 32)
        self.inception_3b = InceptionModule(256, 128, 128, 192, 32, 96, 64)
        self.inception_4a = InceptionModule(480, 192, 96, 208, 16, 48, 64)
        self.inception_4b = InceptionModule(512, 160, 112, 224, 24, 64, 64)
        self.inception_4c = InceptionModule(512, 128, 128, 256, 24, 64, 64)
        self.inception_4d = InceptionModule(512, 112, 144, 288, 32, 64, 64)
        self.inception_4e = InceptionModule(528, 256, 160, 320, 32, 128, 128)
        self.inception_5a = InceptionModule(832, 256, 160, 320, 32, 128, 128)
        self.inception_5b = InceptionModule(832, 384, 192, 384, 48, 128, 128)

        self.aux_classifier1 = AuxClassifier(512, num_class)
        self.aux_classifier2 = AuxClassifier(528, num_class)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(7, 7))
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.4),
            nn.Linear(1024 * 7 * 7, num_class)
        )

    def forward(self, input_img):
        N = input_img.shape[0]
        x = self.conv1(input_img)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool1(x)
        x = self.inception_3a(x)
        x = self.inception_3b(x)
        x = self.maxpool1(x)
        x = self.inception_4a(x)
        out1 = self.aux_classifier1(x)
        x = self.inception_4b(x)
        x = self.inception_4c(x)
        x = self.inception_4d(x)
        out2 = self.aux_classifier2(x)
        x = self.inception_4e(x)
        x = self.maxpool1(x)
        x = self.inception_5a(x)
        x = self.inception_5b(x)
        x = self.avgpool(x)
        x = x.reshape(N, -1)
        x = self.classifier(x)
        if self.training == True:
            return [x, out1, out2]
        else:
            return x

### **4.Train and logging**

In [ ]:
class GoogLeNet_PL(pl.LightningModule):
  def __init__(self):
    super().__init__()
    # define model and loss
    self.model = GoogleNet()
    self.loss = nn.CrossEntropyLoss()

  def forward(self, x):
    return self.model(x)
  
  def custom_histogram_adder(self):
    for name,params in self.named_parameters():
      self.logger.experiment.add_histogram(name,params,self.current_epoch)
  
  def training_step(self, batch, batch_no):
    x, y = batch

    # GoogLeNet training loss from auxiliary classifiers
    outputs, aux1_outputs, aux2_outputs = self(x)
    discount = 0.3
    loss = (self.loss(outputs, y) + discount * (self.loss(aux1_outputs, y) + self.loss(aux2_outputs, y)))


    pred = self.forward(x)[0]
    correct=pred.argmax(dim=1).eq(y).sum().item()
    total=len(y)
    batch_dictionary={
            #REQUIRED: It ie required for us to return "loss"
            "loss": loss,
            # info to be used at epoch end 
            "correct": correct,
            "total": total
        }
    #self.log('train_loss', loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
    return batch_dictionary
  
  def training_epoch_end(self,outputs):
        #  the function is called after every epoch is completed

        # calculating average loss  
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()

        # calculating correct and total predictions
        correct=sum([x["correct"] for  x in outputs])
        total=sum([x["total"] for  x in outputs])
        self.custom_histogram_adder()
        # creating log dictionary
        self.logger.experiment.add_scalar("Loss/Train",
                                            avg_loss,
                                            self.current_epoch)
        
        self.logger.experiment.add_scalar("Accuracy/Train",
                                            correct/total,
                                            self.current_epoch)

        return None

  def configure_optimizers(self):
    # choose your optimizer
    return torch.optim.Adam(self.parameters(), lr=LEARNING_RATE, weight_decay=5e-4)

In [ ]:
logger = TensorBoardLogger('tensorlogs', name = 'googlenet_CIFAR10Model')
model = GoogLeNet_PL() 
trainer = pl.Trainer(
    tpu_cores=8, # use one GPU
    max_epochs=NUM_EPOCHS, # set number of epochs
    progress_bar_refresh_rate=20,
    logger = logger
)

In [ ]:
trainer.fit(model, train_loader)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir tensorlogs/

### **5.Evaluation**

In [ ]:
def get_prediction(x, model: pl.LightningModule):
  model.freeze() # prepares model for predicting
  probabilities = torch.softmax(model(x), dim=1)
  predicted_class = torch.argmax(probabilities, dim=1)
  return predicted_class, probabilities

In [ ]:
true_y, pred_y = [], []
for batch in tqdm(iter(test_dl), total=len(test_dl)):
  x, y = batch
  true_y.extend(y)
  #print(type(x))
  preds, probs = get_prediction(x, model)
  pred_y.extend(preds.cpu())

In [ ]:
# Test results
print(classification_report(true_y, pred_y, digits=3))

In [ ]:
features, targets = next(iter(test_loader))

features = features
targets = targets

np_features = features[0].numpy()
nhwc_img = np.transpose(np_features, axes=(1, 2, 0))
plt.imshow(nhwc_img)

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
predicted, probability = get_prediction(features[0, None], model)
print('Ground Truth Label',classes[targets[0]])
print('Predicted:', classes[predicted])
print('Probability:', probability[0][predicted]*100)

In [ ]:
!zip -r /content/file.zip /content/tensorlogs/
